# Phylogeny with empress

<a id='setup'></a>

## 1. Setup

In [1]:
from qiime2 import Visualization
import os
import pandas as pd
import numpy as np

import qiime2 as q2

%matplotlib inline
# location of this week's data and all the results produced by this notebook 
# - this should be a path relative to your working directory
data_dir = 'project_data'


if not os.path.isdir(data_dir):
    os.makedirs(data_dir) 

## Import data

In [3]:
#filtered sequence
! wget -nv -O $data_dir/rep-seqs.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza'

# Cleaned Meta data
! wget -nv -O $data_dir/table.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza'

# Taxonomy file generate from silva
! wget -nv -O $data_dir/taxonomy_1.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=taxonomy_1.qza'

2022-11-12 18:30:03 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza [390624/390624] -> "project_data/rep-seqs.qza" [1]
2022-11-12 18:30:04 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza [504534/504534] -> "project_data/table.qza" [1]


## 2. De-novo Phylogeny analysis


### 2.1 Sequence alignment

In [6]:
! qiime alignment mafft \
    --i-sequences $data_dir/rep-seqs.qza \
    --o-alignment $data_dir/aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: project_data/aligned-rep-seqs.qza


### 2.2 Alignment masking

In [8]:
! qiime alignment mask \
    --i-alignment $data_dir/aligned-rep-seqs.qza \
    --o-masked-alignment $data_dir/masked-aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: project_data/masked-aligned-rep-seqs.qza


### 2.3 Tree construction and Visualization

In [9]:
! qiime phylogeny fasttree \
    --i-alignment $data_dir/masked-aligned-rep-seqs.qza \
    --o-tree $data_dir/fasttree-tree.qza

! qiime phylogeny midpoint-root \
    --i-tree $data_dir/fasttree-tree.qza \
    --o-rooted-tree $data_dir/fasttree-tree-rooted.qza

Saved Phylogeny[Unrooted] to: project_data/fasttree-tree.qza
Saved Phylogeny[Rooted] to: project_data/fasttree-tree-rooted.qza


In [10]:
#### 2.3.1 Visualization using qiime2

In [11]:
! qiime empress tree-plot \
    --i-tree $data_dir/fasttree-tree-rooted.qza \
    --m-feature-metadata-file $data_dir/taxonomy_1.qza \
    --o-visualization $data_dir/fasttree-tree-rooted.qzv

Saved Visualization to: project_data/fasttree-tree-rooted.qzv


In [12]:
Visualization.load(f'{data_dir}/fasttree-tree-rooted.qzv')

<visualization: Visualization uuid: b53e509a-d446-4a90-9397-9c1db0235468>

### 2.3 Bootstrapping

In [15]:
! qiime phylogeny raxml-rapid-bootstrap \
    --i-alignment $data_dir/masked-aligned-rep-seqs.qza \
    --p-seed 1723 \
    --p-rapid-bootstrap-seed 9384 \
    --p-bootstrap-replicates 100 \
    --p-substitution-model GTRCAT \
    --p-n-threads 3 \
    --o-tree $data_dir/raxml-cat-bootstrap-tree.qza

^C

Aborted!


In [ ]:
! qiime phylogeny midpoint-root \
    --i-tree $data_dir/raxml-cat-bootstrap-tree.qza \
    --o-rooted-tree $data_dir/raxml-cat-bootstrap-tree-rooted.qza

In [ ]:
! qiime empress tree-plot \
    --i-tree $data_dir/raxml-cat-bootstrap-tree-rooted.qza \
    --m-feature-metadata-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/raxml-cat-bootstrap-tree-rooted.qzv

In [ ]:
Visualization.load(f'{data_dir}/raxml-cat-bootstrap-tree-rooted.qzv')

### 2.4 Fragment insertion

In [18]:
! wget -nv -O $data_dir/sepp-refs-gg-13-8.qza https://data.qiime2.org/2021.4/common/sepp-refs-gg-13-8.qza

2022-11-12 21:44:37 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/sepp-refs-gg-13-8.qza [50161069/50161069] -> "project_data/sepp-refs-gg-13-8.qza" [1]


In [2]:
! qiime fragment-insertion sepp \
    --i-representative-sequences $data_dir/rep-seqs-filtered.qza \
    --i-reference-database $data_dir/sepp-refs-gg-13-8.qza \
    --p-threads 2 \
    --o-tree $data_dir/sepp-tree.qza \
    --o-placements $data_dir/sepp-tree-placements.qza

^C


In [ ]:
! qiime empress tree-plot \
    --i-tree $data_dir/sepp-tree.qza \
    --m-feature-metadata-file $data_dir/taxonomy_1.qza \
    --o-visualization $data_dir/sepp-tree.qzv

In [ ]:
Visualization.load(f'{data_dir}/sepp-tree.qzv')